# A Boolean Retrieval System with Spelling Correction

In [7]:
from functools import total_ordering, reduce
import csv
import re

### Postings

In [8]:
@total_ordering
class Posting:
    
    def __init__(self, docID):
        self._docID = docID
        
    def get_from_corpus(self, corpus):
        return corpus[self._docID]
    
    def __eq__(self, other):
        return self._docID == other._docID
    
    def __gt__(self, other):
        return self._docID > other._docID
    
    def __repr__(self):
        return str(self._docID)

### Posting Lists

In [9]:
class PostingList:
    
    def __init__(self):
        self._postings = []
    
    @classmethod
    def from_docID(cls, docID):
        plist = cls()
        plist._postings = [(Posting(docID))]
        return plist
    
    @classmethod
    def from_posting_list(cls, postingList):
        plist = cls()
        plist._postings = postingList
        return plist
    
    def merge(self, other):
        i = 0
        last = self._postings[-1]
        while (i < len(other._postings) and last == other._postings[i]):
            i += 1
        self._postings += other._postings[i:]
        
    def intersection(self, other):
        intersection = []
        i = 0
        j = 0
        while (i < len(self._postings) and j < len(other._postings)):
            if (self._postings[i] == other._postings[j]):
                intersection.append(self._postings[i])
                i += 1
                j += 1
            elif (self._postings[i] < other._postings[j]):
                i += 1
            else:
                j += 1
        return PostingList.from_posting_list(intersection)
    
    def union(self, other):
        union = []
        i = 0
        j = 0
        while (i < len(self._postings) and j < len(other.postings)):
            if (self._postings[i] == other._postings[j]):
                union.append(self._postings[i])
                i += 1
                j += 1
            elif (self._postings[i] < other._postings[j]):
                union.append(self._postings[i])
                i += 1
            else:
                union.append(other._postings[j])
                j += 1
        for k in range(i, len(self._postings)):
            union.append(self._postings[k])
        for k in range(j, len(other._postings)):
            union.append(other._postings[k])
        return PostingList.from_posting_list(union)
    
    def get_from_corpus(self, corpus):
        return list(map(lambda x: x.get_from_corpus(corpus), self._postings))
    
    def __repr__(self):
        return ", ".join(map(str, self._postings))

### Terms

In [10]:
class ImpossibleMergeError(Exception):
    pass

@total_ordering
class Term:
    
    def __init__(self, term, docID):
        self.term = term
        self.posting_list = PostingList.from_docID(docID)
        
    def merge(self, other):
        if (self.term == other.term):
            self.posting_list.merge(other.posting_list)
        else:
            raise ImpossibleMergeError
    
    def __eq__(self, other):
        return self.term == other.term
    
    def __gt__(self, other):
        return self.term > other.term
    
    def __repr__(self):
        return self.term + ": " + repr(self.posting_list)

### Inverted Index

In [11]:
def normalize(text):
    no_punctuation = re.sub(r'[^\w^\s^-]','',text)
    downcase = no_punctuation.lower()
    return downcase

def tokenize(movie):
    text = normalize(movie.description)
    return list(text.split())

class InvertedIndex:
    
    def __init__(self):
        self._dictionary = []
        
    @classmethod
    def from_corpus(cls, corpus):
        intermediate_dict = {}
        for docID, document in enumerate(corpus):
            tokens = tokenize(document)
            for token in tokens:
                term = Term(token, docID)
                try:
                    intermediate_dict[token].merge(term)
                except KeyError:
                    intermediate_dict[token] = term
            if (docID % 1000 == 0):
                print("ID: " + str(docID))
        idx = cls()
        idx._dictionary = sorted(intermediate_dict.values())
        return idx
    
    def __getitem__(self, key):
        for term in self._dictionary:
            if term.term == key:
                return term.posting_list
        raise KeyError
        
    def __repr__(self):
        return "A dictionary with " + str(len(self._dictionary)) + " terms"

### Reading the Corpus

In [12]:
class MovieDescription:
    
    def __init__(self, title, description):
        self.title = title
        self.description = description
        
    def __repr__(self):
        return self.title
    
def read_movie_descriptions():
    filename = 'data/plot_summaries.txt'
    movie_names_file = 'data/movie.metadata.tsv'
    with open(movie_names_file, 'r') as csv_file:
        movie_names = csv.reader(csv_file, delimiter='\t')
        names_table = {}
        for name in movie_names:
            names_table[name[0]] = name[2]
    with open(filename, 'r') as csv_file:
        descriptions = csv.reader(csv_file, delimiter='\t')
        corpus = []
        for desc in descriptions:
            try:
                movie = MovieDescription(names_table[desc[0]], desc[1])
                corpus.append(movie)
            except KeyError:
                pass
        return corpus

### Putting it all together

In [13]:
class IRsystem:
    
    def __init__(self, corpus, index):
        self._corpus = corpus
        self._index = index
        
    @classmethod
    def from_corpus(cls, corpus):
        index = InvertedIndex.from_corpus(corpus)
        return cls(corpus, index)
    
    def answer_query(self, words): # ['cat', 'batman']
        norm_words = map(normalize, words)
        postings = map(lambda w: self._index[w], norm_words)
        plist = reduce(lambda x, y: x.intersection(y), postings)
        return plist.get_from_corpus(self._corpus)
    
    # New methods, this time with spelling correction!
    def answer_query_sc(self, words):
        norm_words = map(normalize, words)        
        postings = []
        
        # Iterate over each normalized word
        for w in norm_words:
            try:
                # Attempt to retrieve the postings list for the word from the index
                res = self._index[w]
            except KeyError:
                # If the word is not found in the index, spelling correction is attempted
                # Create a list of all words in the dictionary part of the index
                dictionary = [t.term for t in self._index._dictionary]
                # Find the nearest word from the dictionary to the misspelled word
                sub = find_nearest(w, dictionary, keep_first=True)
                # Notify the user of the word correction
                print("{} not found. Did you mean {}?".format(w, sub))
                # Retrieve the postings list for the corrected word
                res = self._index[sub]
            # Add the postings list (original or corrected) to the list of postings
            postings.append(res)
        
        plist = reduce(lambda x, y: x.intersection(y), postings)
        return plist.get_from_corpus(self._corpus)
    
def query(ir, text):
    words = text.split()
    answer = ir.answer_query(words)
    for movie in answer:
        print(movie)
        
def query_sc(ir, text):
    words = text.split()
    answer = ir.answer_query_sc(words)
    for movie in answer:
        print(movie)
    

### Edit Distance

In [14]:
# Define a function that calculates the Levenshtein distance between two strings, u and v.
def edit_distance(u, v):
    # Initialize the number of rows and columns for the dynamic programming matrix.
    # Rows represent characters of the first string, columns represent characters of the second string
    nrows = len(u) + 1
    ncols = len(v) + 1
    
    # Create a matrix with nrows rows and ncols columns initialized to zero.
    M = [[0] * ncols for i in range(0, nrows)]
    
    # Initialize the first column of each row
    # The distance from any prefix of u to an empty string v is the length of the prefix
    for i in range(0, nrows):
        M[i][0] = i
    
    # Initialize the first row of each column
    # The distance from an empty string u to any prefix of v is the length of the prefix
    for j in range(0, ncols):
        M[0][j] = j
    
    # Compute the edit distance between every pair of prefixes of u and v
    for i in range(1, nrows):
        for j in range(1, ncols):
            # Gather candidates for the edit distance:
            # The cost of deletion from u (upward move in the matrix),
            # and the cost of insertion to u (leftward move in the matrix).
            candidates = [M[i-1][j] + 1, M[i][j-1] + 1]
            
            # Check if the characters in both strings at the current indices are equal.
            if u[i-1] == v[j-1]:
                # If they are equal, the previous diagonal element is a candidate.
                candidates.append(M[i-1][j-1])
            else:
                # If not equal, consider the substitution cost (increment diagonal element by 1).
                candidates.append(M[i-1][j-1] + 1)
            
            # The current element is the minimum of the candidates,
            # which represents the optimal operation (insert, delete, or substitute).
            M[i][j] = min(candidates)
    
    # The bottom-right element of the matrix contains the edit distance between the full strings.
    return M[-1][-1]


# Define a function to find the nearest word in the dictionary to the input word.
def find_nearest(word, dictionary, keep_first=False):
    # If keep_first is True, filter the dictionary to only include words
    # that start with the same first letter as the input word.
    if keep_first:
        dictionary = list(filter(lambda w: w[0] == word[0], dictionary))
    
    # Calculate the edit distance between the input word and each word in the dictionary.
    # The edit_distance function presumably calculates the Levenshtein distance.
    # This results in a list of distances.
    distances = map(lambda x: edit_distance(word, x), dictionary)
    
    # Combine each distance with the corresponding word into a list of tuples.
    # Then find the tuple with the smallest distance.
    # The [1] at the end extracts just the word from the tuple, ignoring the distance.
    return min(zip(distances, dictionary))[1]



In [15]:
corpus = read_movie_descriptions()
ir = IRsystem.from_corpus(corpus)

ID: 0
ID: 1000
ID: 2000
ID: 3000
ID: 4000
ID: 5000
ID: 6000
ID: 7000
ID: 8000
ID: 9000
ID: 10000
ID: 11000
ID: 12000
ID: 13000
ID: 14000
ID: 15000
ID: 16000
ID: 17000
ID: 18000
ID: 19000
ID: 20000
ID: 21000
ID: 22000
ID: 23000
ID: 24000
ID: 25000
ID: 26000
ID: 27000
ID: 28000
ID: 29000
ID: 30000
ID: 31000
ID: 32000
ID: 33000
ID: 34000
ID: 35000
ID: 36000
ID: 37000
ID: 38000
ID: 39000
ID: 40000
ID: 41000
ID: 42000


In [16]:
query_sc(ir, "yoida lukke darhth")

yoida not found. Did you mean yoda?
lukke not found. Did you mean luke?
darhth not found. Did you mean darth?
Star Wars Episode V: The Empire Strikes Back
Something, Something, Something Dark Side
Return of the Ewok
Star Wars Episode III: Revenge of the Sith
Star Wars Episode VI: Return of the Jedi
It's a Trap!


In [17]:
query_sc(ir, "lauraa ring")

lauraa not found. Did you mean laura?
Buried Alive
Victim
Huracán Ramírez
Les Vampires
The Good Shepherd
Spartan
Buried Alive II
Sleeping with the Enemy
The Golden Spiders: A Nero Wolfe Mystery
Twin Peaks: Fire Walk With Me
